# Numerical norm bounds for quadrotor 

For a quadrotor system with state $x = \begin{bmatrix}p_x & p_z & \phi & v_x & v_z & \dot{\phi} \end{bmatrix}^T$ we have 

\begin{equation}
\dot{x} = \begin{bmatrix} 
v_x \cos\phi - v_z\sin\phi \\
v_x \sin\phi + v_z\cos\phi \\
\dot{\phi} \\
v_z\dot{\phi} - g\sin{\phi} \\
-v_x\dot{\phi} - g\cos{\phi} + g \\
0
\end{bmatrix}.
\end{equation}

Evaluating the corresponding Jacobian at 0 yields:
\begin{equation}
\nabla f(0)x = \begin{bmatrix} v_x & v_z & \dot{\phi} & -g\phi & 0 & 0 \end{bmatrix}^T
\end{equation}

We want to find an NLDI of the form
\begin{equation}
\dot{x} = \nabla f(0) x + I p, \;\; \|p\| \leq \|Cx\|
\end{equation}

To find $C$, we determine an entry-wise norm bound. That is, for $i=1,\ldots,6$, we want to find $C_i$ such that for all $x$ such that $x_{\text{min}} \leq x \leq x_{\text{max}}$:
\begin{equation}
(\nabla f_i(0)x - \dot{x}_i)^2 \leq x^T C_i x
\end{equation}
and then write
\begin{equation}
\|\dot{x} - \nabla f(0)x\|_2 \leq \|\begin{bmatrix} C_1^{1/2} \\ C_2^{1/2} \\ C_3^{1/2} \\ C_4^{1/2} \\ C_5^{1/2} \\ C_6^{1/2} \end{bmatrix} x \|
\end{equation}


In [401]:
import numpy as np
import cvxpy as cp
import scipy.linalg as sla

In [402]:
g = 9.81

## Define max and min values 

In [403]:
# State is: x = [px, pz, phi, vx, vz, phidot]^T
x_max = np.array([1.1, 1.1, 0.06, 0.5, 1.0, 0.8])
x_min = np.array([-1.1, -1.1, -0.06, -0.5, -1.0, -0.8])

px_max, pz_max, phi_max, vx_max, vz_max, phidot_max = x_max
px_min, pz_min, phi_min, vx_min, vz_min, phidot_min = x_min

n = 6
px_idx, pz_idx, phi_idx, vx_idx, vz_idx, phidot_idx = range(n)

## Find element-wise bounds 

### $f_1$

In [404]:
gridnum = 50
vx = np.linspace(vx_min, vx_max, gridnum)
vz = np.linspace(vz_min, vz_max, gridnum)
phi = np.linspace(phi_min, phi_max, gridnum)

Vx, Vz, Phi = np.meshgrid(vx, vz, phi)

v1 = np.ravel(( Vx - (Vx*np.cos(Phi) - Vz*np.sin(Phi)) )**2)
U1 = np.array([np.ravel(Vx*Vx), 
              np.ravel(Vz*Vz), 
              np.ravel(Phi*Phi),
              2*np.ravel(Vx*Vz), 
              2*np.ravel(Vx*Phi), 
              2*np.ravel(Vz*Phi)]).T

In [405]:
c1 = cp.Variable(6)
cp.Problem(cp.Minimize(cp.max(U1@c1 - v1)), [U1@c1 >= v1, c1[:3]>=0]).solve(verbose=True, solver=cp.MOSEK)



Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 250003          
  Cones                  : 0               
  Scalar variables       : 7               
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 250003          
  Cones                  : 0               
  Scalar variables       : 7               
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer  - threads                : 2               
Optimizer  - solved problem         : the dual        
Optimizer  - Constraints            : 7
Optimizer  - Cones                  : 0
Optimizer  - Scalar variables      

0.0018521669140021692

In [406]:
c1 = c1.value
c1

array([-1.93377422e-18,  1.85289423e-03,  5.14305608e-01, -3.38599051e-19,
       -3.05304174e-18, -2.55127137e-18])

In [407]:
C1 = np.zeros((n,n))
C1[vx_idx, vx_idx] = c1[0]/2
C1[vz_idx, vz_idx] = c1[1]/2
C1[phi_idx, phi_idx] = c1[2]/2
C1[vx_idx, vz_idx] = c1[3]
C1[vx_idx, phi_idx] = c1[4]
C1[vz_idx, phi_idx] = c1[5]
C1 += C1.T

In [408]:
gam1 = np.real(sla.sqrtm(C1))
gam1

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  7.17151036e-01,
         0.00000000e+00,  1.98340877e-16,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -4.25718097e-18,
         0.00000000e+00, -7.86611735e-18,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  9.79354505e-17,
         0.00000000e+00,  4.30452580e-02,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

### $f_2$ 

In [409]:
gridnum = 50
vx = np.linspace(vx_min, vx_max, gridnum)
vz = np.linspace(vz_min, vz_max, gridnum)
phi = np.linspace(phi_min, phi_max, gridnum)

Vx, Vz, Phi = np.meshgrid(vx, vz, phi)

v2 = np.ravel(( Vz - (Vx*np.sin(Phi) + Vz*np.cos(Phi)) )**2)
U2 = np.array([np.ravel(Vx*Vx), 
              np.ravel(Vz*Vz), 
              np.ravel(Phi*Phi),
              2*np.ravel(Vx*Vz), 
              2*np.ravel(Vx*Phi), 
              2*np.ravel(Vz*Phi)]).T

In [410]:
c2 = cp.Variable(6)
cp.Problem(cp.Minimize(cp.max(U2@c2 - v2)), [U2@c2 >= v2, c2[:3]>=0]).solve(verbose=True, solver=cp.MOSEK)



Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 250003          
  Cones                  : 0               
  Scalar variables       : 7               
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 250003          
  Cones                  : 0               
  Scalar variables       : 7               
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer  - threads                : 2               
Optimizer  - solved problem         : the dual        
Optimizer  - Constraints            : 7
Optimizer  - Cones                  : 0
Optimizer  - Scalar variables      

0.0005070396714880391

In [411]:
c2 = c2.value
c2

array([ 2.02881664e-03, -4.76107552e-17,  1.39682569e-01, -2.61109197e-17,
        3.02046084e-16, -5.28702685e-17])

In [412]:
C2 = np.zeros((n,n))
C2[vx_idx, vx_idx] = c2[0]/2
C2[vz_idx, vz_idx] = c2[1]/2
C2[phi_idx, phi_idx] = c2[2]/2
C2[vx_idx, vz_idx] = c2[3]
C2[vx_idx, phi_idx] = c2[4]
C2[vz_idx, phi_idx] = c2[5]
C2 += C2.T

In [413]:
gam2 = np.real(sla.sqrtm(C2))
gam2

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  3.73741313e-01,
         7.21246036e-16, -2.86594840e-17,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  7.21246036e-16,
         4.50423872e-02, -5.79696622e-16,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -5.18670264e-17,
        -5.79696622e-16,  7.46468696e-30,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

### $f_3$ 

No error -- linearization is the same as original

### $f_4$

In [414]:
gridnum = 50
vz = np.linspace(vz_min, vz_max, gridnum)
phi = np.linspace(phi_min, phi_max, gridnum)
phidot = np.linspace(phidot_min, phidot_max, gridnum)

Vz, Phi, Phidot = np.meshgrid(vz, phi, phidot)

v4 = np.ravel(( -g*Phi - (Vz*Phidot - g*np.sin(Phi)) )**2)
U4 = np.array([np.ravel(Vz*Vz), 
              np.ravel(Phi*Phi), 
              np.ravel(Phidot*Phidot),
              2*np.ravel(Vz*Phi), 
              2*np.ravel(Vz*Phidot), 
              2*np.ravel(Phi*Phidot)]).T

In [415]:
c4 = cp.Variable(6)
cp.Problem(cp.Minimize(cp.max(U4@c4 - v4)), [U4@c4 >= v4, c4[:3]>=0]).solve(verbose=True, solver=cp.MOSEK)



Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 250003          
  Cones                  : 0               
  Scalar variables       : 7               
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 250003          
  Cones                  : 0               
  Scalar variables       : 7               
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer  - threads                : 2               
Optimizer  - solved problem         : the dual        
Optimizer  - Constraints            : 7
Optimizer  - Cones                  : 0
Optimizer  - Scalar variables      

0.3201607843659614

In [416]:
c4 = c4.value
c4

array([ 3.20282539e-01,  1.32060869e-11,  5.00441468e-01, -2.83645720e-15,
        9.48479457e-18, -8.97992032e-17])

In [417]:
C4 = np.zeros((n,n))
C4[vz_idx, vz_idx] = c4[0]/2
C4[phi_idx, phi_idx] = c4[1]/2
C4[phidot_idx, phidot_idx] = c4[2]/2
C4[vz_idx, phi_idx] = c4[3]
C4[vz_idx, phidot_idx] = c4[4]
C4[phi_idx, phidot_idx] = c4[5]
C4 += C4.T

In [418]:
gam4 = np.real(sla.sqrtm(C4))
gam4

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  3.63401257e-06,
         0.00000000e+00, -4.90810516e-15,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -4.82227916e-15,
         0.00000000e+00,  5.65935102e-01,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -1.26938572e-16,
         0.00000000e+00,  7.44867077e-18,  7.07418877e-01]])

### $f_5$ 

In [419]:
gridnum = 50
vx = np.linspace(vx_min, vx_max, gridnum)
phi = np.linspace(phi_min, phi_max, gridnum)
phidot = np.linspace(phidot_min, phidot_max, gridnum)

Vx, Phi, Phidot = np.meshgrid(vx, phi, phidot)

v5 = np.ravel(( 0 - (-Vx*Phidot - g*np.cos(Phi) + g) )**2)
U5 = np.array([np.ravel(Vx*Vx), 
              np.ravel(Phi*Phi), 
              np.ravel(Phidot*Phidot),
              2*np.ravel(Vx*Phi), 
              2*np.ravel(Vx*Phidot), 
              2*np.ravel(Phi*Phidot)]).T

In [420]:
c5 = cp.Variable(6)
cp.Problem(cp.Minimize(cp.max(U5@c5 - v5)), [U5@c5 >= v5, c5[:3]>=0]).solve(verbose=True, solver=cp.MOSEK)



Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 250003          
  Cones                  : 0               
  Scalar variables       : 7               
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 250003          
  Cones                  : 0               
  Scalar variables       : 7               
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer  - threads                : 2               
Optimizer  - solved problem         : the dual        
Optimizer  - Constraints            : 7
Optimizer  - Cones                  : 0
Optimizer  - Scalar variables      

0.08367461885009912

In [421]:
c5 = c5.value
c5

array([ 3.34116019e-01,  8.65965879e-02,  1.30514070e-01,  3.44307427e-11,
       -8.83002884e-03,  2.05432232e-11])

In [422]:
C5 = np.zeros((n,n))
C5[vx_idx, vx_idx] = c5[0]/2
C5[phi_idx, phi_idx] = c5[1]/2
C5[phidot_idx, phidot_idx] = c5[2]/2
C5[vx_idx, phi_idx] = c5[3]
C5[vx_idx, phidot_idx] = c5[4]
C5[phi_idx, phidot_idx] = c5[5]
C5 += C5.T

In [423]:
np.linalg.eig(C5)[0]

array([0.33449825, 0.13013184, 0.08659659, 0.        , 0.        ,
       0.        ])

In [424]:
gam5 = np.real(sla.sqrtm(C5))
gam5

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  2.94272982e-01,
         3.98186946e-11,  0.00000000e+00,  3.19149441e-11],
       [ 0.00000000e+00,  0.00000000e+00,  3.98186946e-11,
         5.77951217e-01,  0.00000000e+00, -9.40268883e-03],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  3.19149441e-11,
        -9.40268883e-03,  0.00000000e+00,  3.61144929e-01]])

### $f_6$ 

No error -- linearization is the same as original

## Final system 

In [425]:
from linearize_dynamics import *

In [426]:
A = quadrotor_jacobian(np.zeros(n))
G = np.eye(n)
C = np.vstack([gam1, gam2, gam4, gam5])

### Check correctness 

In [427]:
prop = np.random.random((1000000, n))
rand_xs = x_max*prop + x_min*(1-prop)
fx = xdot_uncontrolled(torch.Tensor(rand_xs))
# print(np.linalg.norm((fx - rand_xs@A.T)@np.linalg.inv(G).T, axis=1) <= np.linalg.norm(rand_xs@C.T, axis=1))
print((np.linalg.norm((fx - rand_xs@A.T)@np.linalg.inv(G).T, axis=1) <= np.linalg.norm(rand_xs@C.T, axis=1)).all())

ratio = np.linalg.norm(rand_xs@C.T, axis=1)/np.linalg.norm((fx - rand_xs@A.T)@np.linalg.inv(G).T, axis=1)
print(ratio.max())
print(ratio.mean())
print(np.median(ratio))

True
2541.4533619098297
3.9740415003721594
2.3853982860700276


### Save 

In [428]:
np.save('A.npy', A)
np.save('G.npy', G)
np.save('C.npy', C)

## Check if robust LQR solves 

In [429]:
import scipy.linalg as la

In [430]:
mass = 1
moment_arm = 0.01
inertia_roll = 15.67e-3

B = np.array([
                [0, 0],
                [0, 0],
                [0, 0],
                [0, 0],
                [1/mass, 1/mass],
                [moment_arm/inertia_roll, -moment_arm/inertia_roll]
            ])
m = B.shape[1]

D = np.zeros((C.shape[0], m))

Q = np.random.randn(n, n)
Q = Q.T @ Q
# Q = np.eye(n)

R = np.random.randn(m, m)
R = R.T @ R
# R = np.eye(m)

In [431]:
alpha = 0.0001

n, m = B.shape
wq = C.shape[0]

S = cp.Variable((n, n), symmetric=True)
Y = cp.Variable((m, n))
mu = cp.Variable()

R_sqrt = la.sqrtm(R)
f = cp.trace(S @ Q) + cp.matrix_frac(Y.T @ R_sqrt, S)

cons_mat = cp.bmat((
    (A @ S + S @ A.T + cp.multiply(mu, G @ G.T) + B @ Y + Y.T @ B.T + alpha * S, S @ C.T + Y.T @ D.T),
    (C @ S + D @ Y, -cp.multiply(mu, np.eye(wq)))
))
cons = [S >> 0, mu >= 1e-2] + [cons_mat << 0]

cp.Problem(cp.Minimize(f), cons).solve(solver=cp.MOSEK, verbose=True)
K = np.linalg.solve(S.value, Y.value.T).T



Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1053            
  Cones                  : 0               
  Scalar variables       : 73              
  Matrix variables       : 3               
  Integer variables      : 0               

Optimizer started.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1053            
  Cones                  : 0               
  Scalar variables       : 73              
  Matrix variables       : 3               
  Integer variables      : 0               

Optimizer  - threads                : 2               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 1000
Optimizer  - Cones                  : 1
Optimizer  - Scalar variable

SolverError: Solver 'MOSEK' failed. Try another solver, or solve with verbose=True for more information.